In [1]:
#####################################################################
#########                 Modelo LightGBM                 ###########
#####################################################################

In [2]:
#Carga de Librerias
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, QuantileTransformer
from sklearn.preprocessing import LabelEncoder
import warnings
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from concurrent.futures import ProcessPoolExecutor
from lightgbm import LGBMRegressor
from scipy.stats import yeojohnson

import torch                                     

# Check if CUDA is available
cuda_available = torch.cuda.is_available()      
print(f"CUDA available: {cuda_available}")      

# Check the device name                         
if cuda_available:                              
    device_name = torch.cuda.get_device_name(0)
    print(f"CUDA device name: {device_name}")

    # Move a tensor to the GPU and print it
    tensor = torch.zeros(1).cuda()
    print(f"Tensor on CUDA: {tensor}")


c:\Users\Usuario\OneDrive\Carpeta ONE DRIVE\Maestria\17 Labo III - Series de Tiempo\Labo3-Guille\ve-Labo3\Lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


CUDA available: True
CUDA device name: NVIDIA GeForce RTX 3060 Laptop GPU
Tensor on CUDA: tensor([0.], device='cuda:0')


In [3]:
#Lectura de datos

#Ventas por customer y periodo
sell_in = pd.read_csv("Datos/sell-in.txt", delimiter="\t")  

#Informacion de los productos
productos_descripcion = pd.read_csv("Datos/tb_productos_descripcion.txt", delimiter="\t")  

#Productos a predecir
productos_a_predecir = pd.read_csv("Datos/productos_a_predecir.txt", delimiter="\t") 

In [4]:
#Primer filtro y Visualizacion

#Filtrar por los 780 productos
filtered_sell_in = sell_in[sell_in['product_id'].isin(productos_a_predecir['product_id'])]

# Contar el número de product_id únicos
num_productos_diferentes = filtered_sell_in['product_id'].nunique()

# Imprimir el resultado
print(f"El número de product_id diferentes es: {num_productos_diferentes}")

print("\n\nDatos de los 780 productos\n")
print(filtered_sell_in.head().to_markdown(index=False, numalign='left', stralign='left'))

print("\n\nDatos Info:\n")
print(filtered_sell_in.info())

# Verificar si hay filas duplicadas en el DataFrame
duplicados = filtered_sell_in.duplicated()

# Contar el número de filas duplicadas
num_duplicados = duplicados.sum()

# Imprimir el número de filas duplicadas
print(f'\n\nHay {num_duplicados} filas duplicadas en el DataFrame.')


El número de product_id diferentes es: 780


Datos de los 780 productos

| periodo   | customer_id   | product_id   | plan_precios_cuidados   | cust_request_qty   | cust_request_tn   | tn      |
|:----------|:--------------|:-------------|:------------------------|:-------------------|:------------------|:--------|
| 201701    | 10234         | 20524        | 0                       | 2                  | 0.053             | 0.053   |
| 201701    | 10032         | 20524        | 0                       | 1                  | 0.13628           | 0.13628 |
| 201701    | 10217         | 20524        | 0                       | 1                  | 0.03028           | 0.03028 |
| 201701    | 10125         | 20524        | 0                       | 1                  | 0.02271           | 0.02271 |
| 201701    | 10012         | 20524        | 0                       | 11                 | 1.54452           | 1.54452 |


Datos Info:

<class 'pandas.core.frame.DataFrame'>
Index: 2293481 entri

In [5]:
#Top 5 productos de los 780 a predecir

# Agrupar por 'product_id', sumar 'tn' y ordenar
product_totals = filtered_sell_in.groupby('product_id')['tn'].sum().sort_values(ascending=False)

# Seleccionar los 5 primeros productos
top_5_products = product_totals.head(5).reset_index()

print(top_5_products)



   product_id           tn
0       20001  50340.39558
1       20002  36337.25439
2       20003  32004.15274
3       20004  24178.15379
4       20005  23191.21852


In [6]:
#Definir funcion para elegir que dataset usar: "780 productos" o "top 5 productos"

def productos_780(filtered_sell_in, top_5_products):
    # Código para cuando el valor es 1
    
    # Contar el número de product_id únicos
    num_productos_diferentes = filtered_sell_in['product_id'].nunique()
    
    # Imprimir el resultado
    print(f"El número de product_id diferentes es: {num_productos_diferentes}")

    print("\n\nDatos de los 780 productos\n")
    print(filtered_sell_in.head().to_markdown(index=False, numalign='left', stralign='left'))

    print("\n\nDatos Info:\n")
    print(filtered_sell_in.info())

    # Contar el número de product_id únicos
    num_productos_diferentes = filtered_sell_in['product_id'].nunique()

    # Verificar si hay filas duplicadas en el DataFrame
    duplicados = filtered_sell_in.duplicated()

    # Contar el número de filas duplicadas
    num_duplicados = duplicados.sum()

    # Imprimir el número de filas duplicadas
    print(f'\n\nHay {num_duplicados} filas duplicadas en el DataFrame.')
    
    data = filtered_sell_in
    print("\n\nFunción para 780 prod ejecutada.")
    return filtered_sell_in # Devuelve el DataFrame filtrado

    

def productos_top5(filtered_sell_in, top_5_products):
    # Código para cuando el valor es 0
        
    filtered_sell_in_top5 = filtered_sell_in[filtered_sell_in['product_id'].isin(top_5_products['product_id'])].reset_index(drop=True)

    # Contar el número de product_id únicos
    num_productos_diferentes = filtered_sell_in_top5['product_id'].nunique()

    # Imprimir el resultado
    print(f"El número de product_id diferentes es: {num_productos_diferentes}")

    print("\n\nDatos de los 5 productos\n")
    print(filtered_sell_in_top5.head().to_markdown(index=False, numalign='left', stralign='left'))

    print("\n\nDatos Info:\n")
    print(filtered_sell_in_top5.info())

    # Contar el número de product_id únicos
    num_productos_diferentes = filtered_sell_in_top5['product_id'].nunique()

    # Verificar si hay filas duplicadas en el DataFrame
    duplicados = filtered_sell_in_top5.duplicated()

    # Contar el número de filas duplicadas
    num_duplicados = duplicados.sum()

    # Imprimir el número de filas duplicadas
    print(f'\n\nHay {num_duplicados} filas duplicadas en el DataFrame.')
    
    data = filtered_sell_in_top5
    print("\n\nFunción para top 5 prod ejecutada.")
    return filtered_sell_in_top5  # Devuelve el DataFrame filtrado
    
    


# Diccionario que actúa como switcher
switcher = {
    1: productos_780,
    0: productos_top5,
}

# Función que usa el switcher (modificada)
def ejecutar_funcion(valor, filtered_sell_in, top_5_products):  
    func = switcher.get(valor, lambda: "Valor inválido")
    return func(filtered_sell_in, top_5_products)  # Pasa los DataFrames a la función


In [7]:
##########################################################################################################################
#############################                 Elegir el dataframe a utilizar                 #############################
##########################################################################################################################

# 1: productos_780,
# 0: productos_top5


data = ejecutar_funcion(1, 
                        filtered_sell_in, 
                        top_5_products)  # Guarda el resultado en 'data'


El número de product_id diferentes es: 780


Datos de los 780 productos

| periodo   | customer_id   | product_id   | plan_precios_cuidados   | cust_request_qty   | cust_request_tn   | tn      |
|:----------|:--------------|:-------------|:------------------------|:-------------------|:------------------|:--------|
| 201701    | 10234         | 20524        | 0                       | 2                  | 0.053             | 0.053   |
| 201701    | 10032         | 20524        | 0                       | 1                  | 0.13628           | 0.13628 |
| 201701    | 10217         | 20524        | 0                       | 1                  | 0.03028           | 0.03028 |
| 201701    | 10125         | 20524        | 0                       | 1                  | 0.02271           | 0.02271 |
| 201701    | 10012         | 20524        | 0                       | 11                 | 1.54452           | 1.54452 |


Datos Info:

<class 'pandas.core.frame.DataFrame'>
Index: 2293481 entri

In [8]:
#Agrupacion de datos mensuales

# Agrupar por periodo y product_id, sumar 'tn' y resetear el índice
sales_by_period_product = (data.groupby(['periodo', 'customer_id','product_id'])['tn'].sum()
                                          .reset_index()
                                          .rename(columns={'tn': 'total_tn'}))

# Imprimir resultados
print("\nVentas totales por periodo y producto:\n")
print(sales_by_period_product.head().to_markdown(index=False, numalign='left', stralign='left'))

print("\n\nDatos Info productos agrupados por mes:\n")
print(sales_by_period_product.info())


Ventas totales por periodo y producto:

| periodo   | customer_id   | product_id   | total_tn   |
|:----------|:--------------|:-------------|:-----------|
| 201701    | 10001         | 20001        | 99.4386    |
| 201701    | 10001         | 20002        | 87.6486    |
| 201701    | 10001         | 20003        | 100.213    |
| 201701    | 10001         | 20004        | 21.7395    |
| 201701    | 10001         | 20006        | 29.172     |


Datos Info productos agrupados por mes:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2293481 entries, 0 to 2293480
Data columns (total 4 columns):
 #   Column       Dtype  
---  ------       -----  
 0   periodo      int64  
 1   customer_id  int64  
 2   product_id   int64  
 3   total_tn     float64
dtypes: float64(1), int64(3)
memory usage: 70.0 MB
None


In [9]:
#Completar y Equilibrar datos de Ventas por Producto, Cliente y Periodo

#Obtener todos los productos y periodos únicos
all_product_ids = sales_by_period_product['product_id'].unique()
all_customers = sales_by_period_product['customer_id'].unique()
all_periods = sales_by_period_product['periodo'].unique()

 
# Crear un DataFrame con todas las combinaciones posibles de product_id y periodo
all_combinations = pd.MultiIndex.from_product([all_product_ids,all_customers, all_periods], names=['product_id','customer_id', 'periodo']).to_frame(index=False)
 
# Realizar el merge con el DataFrame original, llenando los valores faltantes con 0
df_complete = pd.merge(all_combinations, sales_by_period_product, on=['product_id','customer_id', 'periodo'], how='left').fillna(0)
 
# Verificar el resultado
print(df_complete.head().to_markdown(index=False, numalign="left", stralign="left"))

print("\n\nDatos Info")
print(df_complete.info())

# Control de que esté completo
# Contar filas por combinación de customer_id y product_id
rows_per_combination = df_complete.groupby(['customer_id', 'product_id']).size()

# Verificar si todos los valores son 36
if all(rows_per_combination == 36):
    print("\n\nTodos los periodos estan completos.")
else:
    print("\n\nNo todos los estan completos.")

| product_id   | customer_id   | periodo   | total_tn   |
|:-------------|:--------------|:----------|:-----------|
| 20001        | 10001         | 201701    | 99.4386    |
| 20001        | 10001         | 201702    | 198.844    |
| 20001        | 10001         | 201703    | 92.4654    |
| 20001        | 10001         | 201704    | 13.2973    |
| 20001        | 10001         | 201705    | 101.006    |


Datos Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16763760 entries, 0 to 16763759
Data columns (total 4 columns):
 #   Column       Dtype  
---  ------       -----  
 0   product_id   int64  
 1   customer_id  int64  
 2   periodo      int64  
 3   total_tn     float64
dtypes: float64(1), int64(3)
memory usage: 511.6 MB
None


Todos los periodos estan completos.


In [10]:
# Analisis de compras por cliente

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
# 1. Cantidad de veces que un cliente-producto nunca en la historia tuvo venta
nunca_vendidos = df_complete.groupby(['product_id', 'customer_id'])['total_tn'].sum().eq(0).sum()

print(f"1. Cantidad de combinaciones cliente-producto que nunca tuvieron venta: {nunca_vendidos}")

# 2. Cantidad de veces que un cliente dejó de comprar un producto en los últimos 6 periodos
df_complete['periodo'] = pd.to_datetime(df_complete['periodo'], format='%Y%m')
ultimo_periodo = df_complete['periodo'].max()
ultimos_6_periodos = [ultimo_periodo - pd.DateOffset(months=i) for i in range(6)]

def cliente_dejo_comprar(group):
    ultimas_compras = group[group['periodo'].isin(ultimos_6_periodos)]['total_tn']
    return (ultimas_compras.iloc[0] > 0) & (ultimas_compras.iloc[1:] == 0).all()

dejo_comprar = df_complete.sort_values('periodo', ascending=False).groupby(['product_id', 'customer_id'], group_keys=False).apply(cliente_dejo_comprar).sum()

print(f"2. Cantidad de veces que un cliente dejó de comprar un producto en los últimos 6 periodos: {dejo_comprar}")

# 3. Cantidad de veces que un producto empezó a comprarse en los últimos 6 meses
def producto_empezo_comprarse(group):
    compras_historicas = group[group['periodo'] < ultimos_6_periodos[-1]]['total_tn']
    compras_recientes = group[group['periodo'].isin(ultimos_6_periodos)]['total_tn']
    return (compras_historicas == 0).all() & (compras_recientes > 0).any()

empezo_comprarse = df_complete.sort_values('periodo').groupby(['product_id', 'customer_id'], group_keys=False).apply(producto_empezo_comprarse).sum()

print(f"3. Cantidad de veces que un producto empezó a comprarse en los últimos 6 meses: {empezo_comprarse}")

1. Cantidad de combinaciones cliente-producto que nunca tuvieron venta: 202855
2. Cantidad de veces que un cliente dejó de comprar un producto en los últimos 6 periodos: 6189
3. Cantidad de veces que un producto empezó a comprarse en los últimos 6 meses: 21857


In [11]:
# Asumimos que tu DataFrame se llama 'df_complete'

# 1. Eliminar combinaciones cliente-producto que nunca se vendieron
df_vendidos = df_complete.groupby(['product_id', 'customer_id']).filter(lambda x: (x['total_tn'] > 0).any())

print(f"Tamaño original del DataFrame: {len(df_complete)}")
print(f"Tamaño del DataFrame después de eliminar combinaciones sin ventas: {len(df_vendidos)}")

# 2. Identificar y eliminar combinaciones cliente-producto que ya no van a comprar
df_vendidos['periodo'] = pd.to_datetime(df_vendidos['periodo'], format='%Y%m')
ultimo_periodo = df_vendidos['periodo'].max()
ultimos_6_periodos = [ultimo_periodo - pd.DateOffset(months=i) for i in range(6)]

def cliente_sigue_comprando(group):
    ultimas_compras = group[group['periodo'].isin(ultimos_6_periodos)]['total_tn']
    return not ((ultimas_compras.iloc[0] > 0) & (ultimas_compras.iloc[1:] == 0).all())

df_activos = df_vendidos.sort_values('periodo', ascending=False).groupby(['product_id', 'customer_id']).filter(cliente_sigue_comprando)

print(f"Tamaño del DataFrame después de eliminar combinaciones inactivas: {len(df_activos)}")

# Guardar el DataFrame resultante
df_activos.to_csv('df_activos.csv', index=False)
print("DataFrame guardado como 'df_activos.csv'")

Tamaño original del DataFrame: 16763760
Tamaño del DataFrame después de eliminar combinaciones sin ventas: 9460980
Tamaño del DataFrame después de eliminar combinaciones inactivas: 9238176
DataFrame guardado como 'df_activos.csv'


In [12]:
#Ventas por customer y periodo
df_activos2 = pd.read_csv("df_activos.csv")  

print("\n\nDatos de los registros activos\n")
print(df_activos2.head().to_markdown(index=False, numalign='left', stralign='left'))

print("\n\nDatos Info:\n")
print(df_activos2.info())



Datos de los registros activos

| product_id   | customer_id   | periodo    | total_tn   |
|:-------------|:--------------|:-----------|:-----------|
| 21214        | 10550         | 2019-12-01 | 0          |
| 20130        | 10042         | 2019-12-01 | 1.68015    |
| 20130        | 10061         | 2019-12-01 | 0.84008    |
| 20140        | 10174         | 2019-12-01 | 0.0819     |
| 20614        | 10506         | 2019-12-01 | 0          |


Datos Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9238176 entries, 0 to 9238175
Data columns (total 4 columns):
 #   Column       Dtype  
---  ------       -----  
 0   product_id   int64  
 1   customer_id  int64  
 2   periodo      object 
 3   total_tn     float64
dtypes: float64(1), int64(2), object(1)
memory usage: 281.9+ MB
None


In [13]:
#Merge de datos de productos

# Realizar el merge entre `sales_by_period_product` y `productos_descripcion`
merged_df = pd.merge(df_activos2, productos_descripcion, on='product_id', how='left')

# Imprimir el DataFrame resultante
print("\nVentas totales por periodo y producto con descripcion:")
print(merged_df.head().to_markdown(index=False, numalign='left', stralign='left'))

print("\n\nDatos Info")
print(merged_df.info())


Ventas totales por periodo y producto con descripcion:
| product_id   | customer_id   | periodo    | total_tn   | cat1   | cat2           | cat3          | brand    | sku_size   | descripcion   |
|:-------------|:--------------|:-----------|:-----------|:-------|:---------------|:--------------|:---------|:-----------|:--------------|
| 21214        | 10550         | 2019-12-01 | 0          | PC     | DEOS           | RollOn        | NIVEA    | 50         | Aroma 14      |
| 20130        | 10042         | 2019-12-01 | 1.68015    | PC     | CABELLO        | SHAMPOO       | SHAMPOO2 | 930        | Manzana       |
| 20130        | 10061         | 2019-12-01 | 0.84008    | PC     | CABELLO        | SHAMPOO       | SHAMPOO2 | 930        | Manzana       |
| 20140        | 10174         | 2019-12-01 | 0.0819     | PC     | PIEL2          | Jabon Regular | DEOS1    | 125        | Orquideas     |
| 20614        | 10506         | 2019-12-01 | 0          | FOODS  | SOPAS Y CALDOS | Baking Bags  

In [14]:
# Control de Nulos y Ceros

# Contar valores nulos en todo el DataFrame
total_nulos = merged_df.isnull().sum().sum()
print(f'\nHay un total de {total_nulos} valores nulos en el DataFrame.')

# Contar valores nulos por columna
nulos_por_columna = merged_df.isnull().sum()
print('\nValores nulos por columna:\n')
print(nulos_por_columna)

# Contar valores iguales a cero en todo el DataFrame
total_ceros = (merged_df == 0).sum().sum()
print(f'\nHay un total de {total_ceros} valores iguales a cero en el DataFrame.')

# Contar valores iguales a cero por columna
ceros_por_columna = (merged_df == 0).sum()
print('\nValores iguales a cero por columna:')
print(ceros_por_columna)


Hay un total de 0 valores nulos en el DataFrame.

Valores nulos por columna:

product_id     0
customer_id    0
periodo        0
total_tn       0
cat1           0
cat2           0
cat3           0
brand          0
sku_size       0
descripcion    0
dtype: int64

Hay un total de 6982654 valores iguales a cero en el DataFrame.

Valores iguales a cero por columna:
product_id           0
customer_id          0
periodo              0
total_tn       6982654
cat1                 0
cat2                 0
cat3                 0
brand                0
sku_size             0
descripcion          0
dtype: int64


In [15]:
#Declaracion de funciones para el tipo de escalado

def scale_data(df, method='standard'):
    # Crear una copia del DataFrame
    scaled_df = df.copy()
    
    # Agrupar por product_id y customer_id
    grouped = scaled_df.groupby(['product_id', 'customer_id'])
    
    # Función para aplicar el escalado a cada grupo
    def scale_group(group):
        if method == 'standard':
            scaler = StandardScaler()
            group['total_tn_scaled'] = scaler.fit_transform(group[['total_tn']])
        elif method == 'robust':
            scaler = RobustScaler()
            group['total_tn_scaled'] = scaler.fit_transform(group[['total_tn']])
        elif method == 'log':
            group['total_tn_scaled'] = np.log1p(group['total_tn'])
        elif method == 'yeo-johnson':
            group['total_tn_scaled'], _ = yeojohnson(group['total_tn'])
        elif method == 'quantile':
            scaler = QuantileTransformer(output_distribution='normal')
            group['total_tn_scaled'] = scaler.fit_transform(group[['total_tn']])
        else:
            raise ValueError("Método de escalado no reconocido")
        
        return group

    # Aplicar el escalado a cada grupo
    scaled_df = grouped.apply(scale_group).reset_index(drop=True)
    
    return scaled_df

In [16]:
#######################################################################################################################################################
############################################                 Elegir el escalado a utilizar                 ############################################
#######################################################################################################################################################

#Los métodos disponibles en este switcher son:

#  standard (StandardScaler)
#  robust (RobustScaler)
#  log (Transformación logarítmica)
#  yeo-johnson (PowerTransformer con método Yeo-Johnson)
#  quantile (QuantileTransformer con distribución normal)

# Crear una copia del DataFrame
data_scaled = merged_df.copy()

# Ejemplo de uso
method = 'log'  # Cambia este valor a 'standard', 'robust', 'log', 'yeo-johnson', o 'quantile' para probar diferentes métodos
data_scaled = scale_data(data_scaled, method)

# Imprimir el DataFrame resultante
print("\nVentas escaladas por periodo y producto con descripcion:")
print(data_scaled.head().to_markdown(index=False, numalign='left', stralign='left'))

print("\n\nDatos Info")
print(data_scaled.info())

print("\n\nMetodo de escalado:")
print(method)


Ventas escaladas por periodo y producto con descripcion:
| product_id   | customer_id   | periodo    | total_tn   | cat1   | cat2        | cat3    | brand   | sku_size   | descripcion   | total_tn_scaled   |
|:-------------|:--------------|:-----------|:-----------|:-------|:------------|:--------|:--------|:-----------|:--------------|:------------------|
| 20001        | 10001         | 2019-12-01 | 180.219    | HC     | ROPA LAVADO | Liquido | ARIEL   | 3000       | genoma        | 5.19971           |
| 20001        | 10001         | 2019-11-01 | 236.656    | HC     | ROPA LAVADO | Liquido | ARIEL   | 3000       | genoma        | 5.47082           |
| 20001        | 10001         | 2019-10-01 | 176.03     | HC     | ROPA LAVADO | Liquido | ARIEL   | 3000       | genoma        | 5.17632           |
| 20001        | 10001         | 2019-09-01 | 109.052    | HC     | ROPA LAVADO | Liquido | ARIEL   | 3000       | genoma        | 4.70096           |
| 20001        | 10001         | 201

In [17]:
# Control de Nulos y Ceros

# Contar valores nulos en todo el DataFrame
total_nulos = data_scaled.isnull().sum().sum()
print(f'\nHay un total de {total_nulos} valores nulos en el DataFrame.')

# Contar valores nulos por columna
nulos_por_columna = data_scaled.isnull().sum()
print('\nValores nulos por columna:\n')
print(nulos_por_columna)

# Contar valores iguales a cero en todo el DataFrame
total_ceros = (data_scaled == 0).sum().sum()
print(f'\nHay un total de {total_ceros} valores iguales a cero en el DataFrame.')

# Contar valores iguales a cero por columna
ceros_por_columna = (data_scaled == 0).sum()
print('\nValores iguales a cero por columna:')
print(ceros_por_columna)


Hay un total de 0 valores nulos en el DataFrame.

Valores nulos por columna:

product_id         0
customer_id        0
periodo            0
total_tn           0
cat1               0
cat2               0
cat3               0
brand              0
sku_size           0
descripcion        0
total_tn_scaled    0
dtype: int64

Hay un total de 13965308 valores iguales a cero en el DataFrame.

Valores iguales a cero por columna:
product_id               0
customer_id              0
periodo                  0
total_tn           6982654
cat1                     0
cat2                     0
cat3                     0
brand                    0
sku_size                 0
descripcion              0
total_tn_scaled    6982654
dtype: int64


In [18]:
#Agregacion de nuevas columnas al dataset

# 1. Ordenar el DataFrame
df = data_scaled.sort_values(['product_id', 'periodo', 'customer_id'])

# 2. Agregar LAGS

# Definir el número de lags
num_lags = 12  # Puedes ajustar este valor según tus necesidades

for i in range(1, num_lags+1): 
    df[f'total_tn_lag_{i}'] = df.groupby(['product_id', 'customer_id'])['total_tn_scaled'].shift(i)

# 3. Agregar deltas de los LAGS
for i in range(1, 13):
    df[f'delta_total_tn_lag_{i}'] = df['total_tn_scaled'] - df[f'total_tn_lag_{i}']

# 4. Agregar total_tn_estandarizado n+2
df['target'] = df.groupby(['product_id', 'customer_id'])['total_tn_scaled'].shift(-2)

# 5. Agregar año, mes y quarter
df['periodo'] = pd.to_datetime(df['periodo'])
df['año'] = df['periodo'].dt.year
df['mes'] = df['periodo'].dt.month
df['quarter'] = df['periodo'].dt.quarter

# 6. Crear parent_product
df['parent_product'] = df.groupby(['cat1', 'cat2', 'cat3', 'brand']).ngroup()

# 7. Crear variables que indiquen la posición relativa del producto dentro de su familia
df['sku_size_rank'] = df.groupby('parent_product')['sku_size'].rank(method='dense')
df['is_smallest_sku'] = df['sku_size_rank'] == 1
df['is_largest_sku'] = df.groupby('parent_product')['sku_size_rank'].transform('max') == df['sku_size_rank']

# 8. Calcular la participación de mercado de cada SKU dentro de su parent_product
df['parent_product_sales'] = df.groupby(['parent_product', 'periodo'])['total_tn_scaled'].transform('sum')
df['market_share_in_parent'] = df['total_tn_scaled'] / df['parent_product_sales']
df['avg_market_share_in_parent'] = df.groupby(['parent_product', 'product_id'])['market_share_in_parent'].transform('mean')

# Mostrar las primeras filas del DataFrame procesado
print("\n\nColumnas actuales\n")
print(df[['product_id','customer_id', 'parent_product', 'sku_size', 'sku_size_rank', 'is_smallest_sku', 'is_largest_sku', 
           'market_share_in_parent', 'avg_market_share_in_parent']].head(5).to_markdown(index=False, numalign='left', stralign='left'))

# Mostrar información sobre el DataFrame procesado
print("\n\nDatos Info\n")
print(df.info())



Columnas actuales

| product_id   | customer_id   | parent_product   | sku_size   | sku_size_rank   | is_smallest_sku   | is_largest_sku   | market_share_in_parent   | avg_market_share_in_parent   |
|:-------------|:--------------|:-----------------|:-----------|:----------------|:------------------|:-----------------|:-------------------------|:-----------------------------|
| 20001        | 10001         | 56               | 3000       | 5               | False             | True             | 0.0215242                | 0.00113373                   |
| 20001        | 10002         | 56               | 3000       | 5               | False             | True             | 0.0168267                | 0.00113373                   |
| 20001        | 10003         | 56               | 3000       | 5               | False             | True             | 0.0232225                | 0.00113373                   |
| 20001        | 10004         | 56               | 3000       | 5             

In [19]:
# Control de Nulos y Ceros

# Contar valores nulos en todo el DataFrame
total_nulos = df.isnull().sum().sum()
print(f'\nHay un total de {total_nulos} valores nulos en el DataFrame.')

# Contar valores nulos por columna
nulos_por_columna = df.isnull().sum()
print('\nValores nulos por columna:\n')
print(nulos_por_columna)

# Contar valores iguales a cero en todo el DataFrame
total_ceros = (df == 0).sum().sum()
print(f'\nHay un total de {total_ceros} valores iguales a cero en el DataFrame.')

# Contar valores iguales a cero por columna
ceros_por_columna = (df == 0).sum()
print('\nValores iguales a cero por columna:')
print(ceros_por_columna)


Hay un total de 40956736 valores nulos en el DataFrame.

Valores nulos por columna:

product_id                          0
customer_id                         0
periodo                             0
total_tn                            0
cat1                                0
cat2                                0
cat3                                0
brand                               0
sku_size                            0
descripcion                         0
total_tn_scaled                     0
total_tn_lag_1                 256616
total_tn_lag_2                 513232
total_tn_lag_3                 769848
total_tn_lag_4                1026464
total_tn_lag_5                1283080
total_tn_lag_6                1539696
total_tn_lag_7                1796312
total_tn_lag_8                2052928
total_tn_lag_9                2309544
total_tn_lag_10               2566160
total_tn_lag_11               2822776
total_tn_lag_12               3079392
delta_total_tn_lag_1           256616
de

In [20]:
print(df.tail().to_markdown(index=False, numalign='left', stralign='left'))

| product_id   | customer_id   | periodo             | total_tn   | cat1   | cat2   | cat3   | brand   | sku_size   | descripcion    | total_tn_scaled   | total_tn_lag_1   | total_tn_lag_2   | total_tn_lag_3   | total_tn_lag_4   | total_tn_lag_5   | total_tn_lag_6   | total_tn_lag_7   | total_tn_lag_8   | total_tn_lag_9   | total_tn_lag_10   | total_tn_lag_11   | total_tn_lag_12   | delta_total_tn_lag_1   | delta_total_tn_lag_2   | delta_total_tn_lag_3   | delta_total_tn_lag_4   | delta_total_tn_lag_5   | delta_total_tn_lag_6   | delta_total_tn_lag_7   | delta_total_tn_lag_8   | delta_total_tn_lag_9   | delta_total_tn_lag_10   | delta_total_tn_lag_11   | delta_total_tn_lag_12   | target   | año   | mes   | quarter   | parent_product   | sku_size_rank   | is_smallest_sku   | is_largest_sku   | parent_product_sales   | market_share_in_parent   | avg_market_share_in_parent   |
|:-------------|:--------------|:--------------------|:-----------|:-------|:-------|:-------|:--------|:--------

In [21]:
# Suponiendo que 'target' es la columna donde quieres verificar los NaN
periodos_con_nan = df[df['target'].isna()]['periodo'].unique()

print("Periodos con NaN en 'target':")
print(periodos_con_nan)

Periodos con NaN en 'target':
<DatetimeArray>
['2019-11-01 00:00:00', '2019-12-01 00:00:00']
Length: 2, dtype: datetime64[ns]


In [22]:
# Contar el número de product_id únicos
num_productos_diferentes = df['product_id'].nunique()

# Imprimir el resultado
print(f"El número de product_id diferentes es: {num_productos_diferentes}")

El número de product_id diferentes es: 780


In [23]:
# Selecciona las características y el target
features = [col for col in df.columns if col not in ['descripcion']]
data = df[features]

# Parámetros de división
test_months = 2 
val_months = 2 

# Fechas de corte (Usamos la columna 'fecha' del dataset)
test_start_date = data['periodo'].max() - pd.DateOffset(months=test_months)
val_start_date = test_start_date - pd.DateOffset(months=val_months)

# Máscaras para seleccionar conjuntos 
train_mask = data['periodo'] < val_start_date
val_mask = (data['periodo'] >= val_start_date) & (data['periodo'] < test_start_date)
test_mask = data['periodo'] >= test_start_date

# División de datos
X_train = data.drop(['target', 'periodo', 'total_tn'], axis=1)[train_mask]
y_train = data['target'][train_mask]
weights_train = data['total_tn'][train_mask]

X_val = data.drop(['target', 'periodo', 'total_tn'], axis=1)[val_mask]
y_val = data['target'][val_mask]
weights_val = data['total_tn'][val_mask]

X_test = data.drop(['target', 'periodo', 'total_tn'], axis=1)[test_mask]
y_test = data['target'][test_mask]
weights_test = data['total_tn'][test_mask]

prediction_data = data[test_mask].copy()

# Columnas categóricas

# Columnas categóricas tus DataFrames X_train, X_val, X_test
categorical_cols = ['cat1', 'cat2', 'cat3', 'brand']

# Crear un codificador para cada columna categórica
encoders = {}
for col in categorical_cols:
    encoders[col] = LabelEncoder()
    X_train[col] = encoders[col].fit_transform(X_train[col])

# Transformar los conjuntos de validación y prueba (usando los mismos codificadores)
for col in categorical_cols:
    X_val[col] = encoders[col].transform(X_val[col])
    X_test[col] = encoders[col].transform(X_test[col])
    prediction_data[col] = encoders[col].transform(prediction_data[col])
 



In [24]:
print(X_test.head().to_markdown(index=False, numalign='left', stralign='left'))

| product_id   | customer_id   | cat1   | cat2   | cat3   | brand   | sku_size   | total_tn_scaled   | total_tn_lag_1   | total_tn_lag_2   | total_tn_lag_3   | total_tn_lag_4   | total_tn_lag_5   | total_tn_lag_6   | total_tn_lag_7   | total_tn_lag_8   | total_tn_lag_9   | total_tn_lag_10   | total_tn_lag_11   | total_tn_lag_12   | delta_total_tn_lag_1   | delta_total_tn_lag_2   | delta_total_tn_lag_3   | delta_total_tn_lag_4   | delta_total_tn_lag_5   | delta_total_tn_lag_6   | delta_total_tn_lag_7   | delta_total_tn_lag_8   | delta_total_tn_lag_9   | delta_total_tn_lag_10   | delta_total_tn_lag_11   | delta_total_tn_lag_12   | año   | mes   | quarter   | parent_product   | sku_size_rank   | is_smallest_sku   | is_largest_sku   | parent_product_sales   | market_share_in_parent   | avg_market_share_in_parent   |
|:-------------|:--------------|:-------|:-------|:-------|:--------|:-----------|:------------------|:-----------------|:-----------------|:-----------------|:----------------

In [25]:
print(prediction_data.head().to_markdown(index=False, numalign='left', stralign='left'))

| product_id   | customer_id   | periodo             | total_tn   | cat1   | cat2   | cat3   | brand   | sku_size   | total_tn_scaled   | total_tn_lag_1   | total_tn_lag_2   | total_tn_lag_3   | total_tn_lag_4   | total_tn_lag_5   | total_tn_lag_6   | total_tn_lag_7   | total_tn_lag_8   | total_tn_lag_9   | total_tn_lag_10   | total_tn_lag_11   | total_tn_lag_12   | delta_total_tn_lag_1   | delta_total_tn_lag_2   | delta_total_tn_lag_3   | delta_total_tn_lag_4   | delta_total_tn_lag_5   | delta_total_tn_lag_6   | delta_total_tn_lag_7   | delta_total_tn_lag_8   | delta_total_tn_lag_9   | delta_total_tn_lag_10   | delta_total_tn_lag_11   | delta_total_tn_lag_12   | target   | año   | mes   | quarter   | parent_product   | sku_size_rank   | is_smallest_sku   | is_largest_sku   | parent_product_sales   | market_share_in_parent   | avg_market_share_in_parent   |
|:-------------|:--------------|:--------------------|:-----------|:-------|:-------|:-------|:--------|:-----------|:------------

In [26]:
train_data = lgb.Dataset(X_train, label=y_train, weight=weights_train, feature_name=list(X_train.columns), categorical_feature=categorical_cols)
val_data = lgb.Dataset(X_val, label=y_val, weight=weights_val, feature_name=list(X_val.columns), categorical_feature=categorical_cols)

In [27]:
params = {
    'objective': 'regression',  # Tipo de problema: 'regression', 'binary', 'multiclass'
    'metric': ['mse', 'huber'],  # Métricas de evaluación. Opciones: 'mse', 'rmse', 'mae', 'mape', 'huber'
    'num_leaves': 283,  # Número máximo de hojas en cada árbol. Rango típico: 20-100
    'max_depth': -1,  # Profundidad máxima del árbol. -1 para sin límite. Rango típico: 3-12
    'learning_rate': 0.008,  # Tasa de aprendizaje. Rango típico: 0.01-0.3
    'feature_fraction': 0.67,  # Fracción de características usadas en cada iteración. Rango: 0.5-1.0
    'bagging_fraction': 0.88,  # Fracción de datos usados para cada iteración. Rango: 0.5-1.0
    'bagging_freq': 8,  # Frecuencia de bagging. 0 significa deshabilitar bagging. Valor típico: 5
    'lambda_l1': 0.21, 
    'lambda_l2': 0.67, 
    'min_child_samples': 36,  # Número mínimo de muestras en un nodo hoja. Rango típico: 10-50
    'reg_alpha': 0.01,  # Regularización L1. Rango típico: 0-1
    'reg_lambda': 0.01,  # Regularización L2. Rango típico: 0-1
    'min_split_gain': 0.1,  # Ganancia mínima para realizar una división. Rango típico: 0-0.5
    'max_bin': 255,  # Número máximo de bins para variables continuas. Rango típico: 200-1000
    'boosting': 'gbdt',  # Tipo de boosting. Opciones: 'gbdt', 'dart', 'goss'
    'verbose': -1,  # Nivel de detalle en la salida. -1: nada, 0: errores, 1: advertencias, >1: info
    'gpu_platform_id': 0,
    'gpu_device_id': 1,
    'device': 'gpu',  # Dispositivo a utilizar: 'cpu' o 'gpu'
}

num_boost_round = 1000  # Número máximo de iteraciones de boosting

# Callbacks para early stopping y logging
callbacks = [
    lgb.early_stopping(stopping_rounds=50),  # Detiene el entrenamiento si no hay mejora en 50 rondas
    lgb.log_evaluation(period=100)  # Registra la evaluación cada 100 iteraciones
]

model = lgb.train(
    params, 
    train_data, 
    num_boost_round, 
    valid_sets=[val_data],
    callbacks=callbacks
)

Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 1.06872	valid_0's huber: 0.436037
[200]	valid_0's l2: 0.768422	valid_0's huber: 0.322988
[300]	valid_0's l2: 0.689983	valid_0's huber: 0.288922
[400]	valid_0's l2: 0.66505	valid_0's huber: 0.277348
[500]	valid_0's l2: 0.655874	valid_0's huber: 0.27301
[600]	valid_0's l2: 0.651108	valid_0's huber: 0.270615
[700]	valid_0's l2: 0.648765	valid_0's huber: 0.269281
[800]	valid_0's l2: 0.647382	valid_0's huber: 0.268353
Early stopping, best iteration is:
[847]	valid_0's l2: 0.647105	valid_0's huber: 0.268171


In [28]:
# Predicciones en el conjunto de validación
val_preds = model.predict(X_val, num_iteration=model.best_iteration)

# Calcula métricas
mse = mean_squared_error(y_val, val_preds)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, val_preds)

print(f"RMSE: {rmse}")
print(f"R2 Score: {r2}")

RMSE: 0.1463668165429294
R2 Score: 0.6065366651089463


In [29]:
prediction_data_final = prediction_data[prediction_data['periodo'] == '2019-12-01 00:00:00'].copy()
print(prediction_data_final.tail().to_markdown(index=False, numalign='left', stralign='left'))

| product_id   | customer_id   | periodo             | total_tn   | cat1   | cat2   | cat3   | brand   | sku_size   | total_tn_scaled   | total_tn_lag_1   | total_tn_lag_2   | total_tn_lag_3   | total_tn_lag_4   | total_tn_lag_5   | total_tn_lag_6   | total_tn_lag_7   | total_tn_lag_8   | total_tn_lag_9   | total_tn_lag_10   | total_tn_lag_11   | total_tn_lag_12   | delta_total_tn_lag_1   | delta_total_tn_lag_2   | delta_total_tn_lag_3   | delta_total_tn_lag_4   | delta_total_tn_lag_5   | delta_total_tn_lag_6   | delta_total_tn_lag_7   | delta_total_tn_lag_8   | delta_total_tn_lag_9   | delta_total_tn_lag_10   | delta_total_tn_lag_11   | delta_total_tn_lag_12   | target   | año   | mes   | quarter   | parent_product   | sku_size_rank   | is_smallest_sku   | is_largest_sku   | parent_product_sales   | market_share_in_parent   | avg_market_share_in_parent   |
|:-------------|:--------------|:--------------------|:-----------|:-------|:-------|:-------|:--------|:-----------|:------------

In [30]:
X_pred = prediction_data_final.drop(['target','periodo', 'total_tn'], axis=1)

predictions = model.predict(X_pred, num_iteration=model.best_iteration)

# Añade las predicciones al DataFrame de predicción
prediction_data_final['predicted_tn'] = predictions

In [31]:
print(prediction_data_final.head().to_markdown(index=False, numalign='left', stralign='left'))

| product_id   | customer_id   | periodo             | total_tn   | cat1   | cat2   | cat3   | brand   | sku_size   | total_tn_scaled   | total_tn_lag_1   | total_tn_lag_2   | total_tn_lag_3   | total_tn_lag_4   | total_tn_lag_5   | total_tn_lag_6   | total_tn_lag_7   | total_tn_lag_8   | total_tn_lag_9   | total_tn_lag_10   | total_tn_lag_11   | total_tn_lag_12   | delta_total_tn_lag_1   | delta_total_tn_lag_2   | delta_total_tn_lag_3   | delta_total_tn_lag_4   | delta_total_tn_lag_5   | delta_total_tn_lag_6   | delta_total_tn_lag_7   | delta_total_tn_lag_8   | delta_total_tn_lag_9   | delta_total_tn_lag_10   | delta_total_tn_lag_11   | delta_total_tn_lag_12   | target   | año   | mes   | quarter   | parent_product   | sku_size_rank   | is_smallest_sku   | is_largest_sku   | parent_product_sales   | market_share_in_parent   | avg_market_share_in_parent   | predicted_tn   |
|:-------------|:--------------|:--------------------|:-----------|:-------|:-------|:-------|:--------|:--------

In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, QuantileTransformer

def inverse_scale_data(df, original_df, method):
    def inverse_scale_group(group, original_group):
        if method == 'standard' or method == 'robust' or method == 'quantile':
            # Usar la relación entre los datos originales y escalados
            scale_factor = original_group['total_tn'].mean() / group['total_tn_scaled'].mean()
            return group['predicted_tn'] * scale_factor
        elif method == 'log':
            return np.expm1(group['predicted_tn'])
        elif method == 'yeo-johnson':
            pt = PowerTransformer(method='yeo-johnson')
            pt.fit(original_group['total_tn'].values.reshape(-1, 1))
            return pt.inverse_transform(group['predicted_tn'].values.reshape(-1, 1)).flatten()
        else:
            raise ValueError(f"Unknown method: {method}")

    # Aplicar la transformación inversa a cada grupo
    df['target_unscaled'] = df.groupby(['customer_id', 'product_id'])['predicted_tn'].transform(
        lambda x: np.expm1(x) if method == 'log' else x
    )
    return df

# Asumiendo que 'method' es el método que usaste para escalar originalmente
#method = 'standard'  # Cambia esto al método que hayas usado

print("Metodo de des-escalado:")
print(method)

# Aplicar la transformación inversa
prediction_data = inverse_scale_data(prediction_data_final, data_scaled, method)

# Agrupar por ID y sumar los valores desnormalizados
result = prediction_data.groupby('product_id')['target_unscaled'].sum().reset_index()

# Renombrar la columna 'target_unscaled' a 'tn'
result = result.rename(columns={'target_unscaled': 'tn'})

# Guardar el resultado en un CSV
result.to_csv('target_unscaled_sum_by_id_12_07_v9.csv', index=False)

print("\n¡Archivo listo!")

Metodo de des-escalado:
log

¡Archivo listo!


In [33]:
feature_importance = model.feature_importance()
feature_names = model.feature_name()

# Crea un DataFrame con la importancia de las características
feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print(feature_importance_df.head(10))

                   feature  importance
1              customer_id       11978
20    delta_total_tn_lag_1        9536
21    delta_total_tn_lag_2        9055
40  market_share_in_parent        8576
22    delta_total_tn_lag_3        8286
23    delta_total_tn_lag_4        8084
39    parent_product_sales        8001
24    delta_total_tn_lag_5        7458
8           total_tn_lag_1        7195
25    delta_total_tn_lag_6        7193


In [34]:
#########################################################
####################   RadomSearch   ####################
#########################################################

In [35]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.model_selection import PredefinedSplit

# Definir el espacio de búsqueda de hiperparámetros (esto permanece igual)
param_dist = {
    'num_leaves': randint(30, 300),
    'learning_rate': uniform(0.01, 0.8),
    'feature_fraction': uniform(0.8, 0.4),
    'bagging_fraction': uniform(0.8, 0.4),
    'bagging_freq': randint(1, 10),
    'min_child_samples': randint(5, 100),
    'lambda_l1': uniform(0, 1),
    'lambda_l2': uniform(0, 1)
}

# Crear el modelo base
base_model = lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    device='gpu',
    gpu_platform_id=0,
    gpu_device_id=1,
    verbose=-1
)

# Combinar datos de entrenamiento y validación
X_combined = pd.concat([X_train, X_val])
y_combined = pd.concat([y_train, y_val])
weights_combined = pd.concat([weights_train, weights_val])

# Crear un array de índices para PredefinedSplit
test_fold = [-1] * len(X_train) + [0] * len(X_val)
ps = PredefinedSplit(test_fold)

# Configurar y ejecutar la búsqueda aleatoria
random_search = RandomizedSearchCV(
    base_model,
    param_distributions=param_dist,
    n_iter=100,
    cv=ps,
    random_state=42,
    n_jobs=2
)

# Definir fit_params para incluir los pesos y el conjunto de validación
fit_params = {
    "sample_weight": weights_combined,
    "eval_set": [(X_val, y_val)],
    "eval_sample_weight": [weights_val],
    "callbacks": [lgb.early_stopping(stopping_rounds=50)]
}

# Ajustar el modelo
random_search.fit(X_combined, y_combined, **fit_params)

# Obtener el mejor modelo
best_model = random_search.best_estimator_

# Realizar predicciones y evaluar
val_preds = best_model.predict(X_val)

mse = mean_squared_error(y_val, val_preds)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, val_preds)

print(f"RMSE: {rmse}")
print(f"R2 Score: {r2}")
print("Mejores parámetros:", random_search.best_params_)

c:\Users\Usuario\OneDrive\Carpeta ONE DRIVE\Maestria\17 Labo III - Series de Tiempo\Labo3-Guille\ve-Labo3\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
c:\Users\Usuario\OneDrive\Carpeta ONE DRIVE\Maestria\17 Labo III - Series de Tiempo\Labo3-Guille\ve-Labo3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
36 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Usuario\OneDrive\Carpeta ONE DRIVE\Maestria\17 Lab

Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 0.493855
RMSE: 0.13911906837115037
R2 Score: 0.6445386910477233
Mejores parámetros: {'bagging_fraction': 0.8125716742746938, 'bagging_freq': 7, 'feature_fraction': 0.9901480892728447, 'lambda_l1': 0.5632755719763837, 'lambda_l2': 0.6955160864261275, 'learning_rate': 0.12146516352470055, 'min_child_samples': 19, 'num_leaves': 200}
